
# Data Cleaning and Manipulation

This notebook will be used for gathering data from our primary sources and transforming it into a format that can be utilized for machine learning applications

In [81]:
# import libraries
import pandas as pd
from functools import reduce
import glob

In [82]:
# read all data
crdc_dfs = [pd.read_csv(filename, encoding='ANSI', true_values=['Yes'], false_values=['No'], na_values=['-3','-5','-6','-8','-9','-11'], dtype={'LEAID':str,'COMBOKEY':str,'SCHID':str}) for filename in glob.glob("../data/2017-18-crdc-data/Data/SCH/CRDC/CSV/*.csv")]

C:\Users\David\AppData\Local\Temp\ipykernel_11560\2738817395.py:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  crdc_dfs = [pd.read_csv(filename, encoding='ANSI', true_values=['Yes'], false_values=['No'], na_values=['-3','-5','-6','-8','-9','-11'], dtype={'LEAID':str,'COMBOKEY':str,'SCHID':str}) for filename in glob.glob("../data/2017-18-crdc-data/Data/SCH/CRDC/CSV/*.csv")]
C:\Users\David\AppData\Local\Temp\ipykernel_11560\2738817395.py:2: DtypeWarning: Columns (8,22) have mixed types.Specify dtype option on import or set low_memory=False.
  crdc_dfs = [pd.read_csv(filename, encoding='ANSI', true_values=['Yes'], false_values=['No'], na_values=['-3','-5','-6','-8','-9','-11'], dtype={'LEAID':str,'COMBOKEY':str,'SCHID':str}) for filename in glob.glob("../data/2017-18-crdc-data/Data/SCH/CRDC/CSV/*.csv")]


In [83]:
# merge dfs together
merge_cols = ['LEA_STATE','LEA_STATE_NAME','LEAID','LEA_NAME','SCHID','SCH_NAME','COMBOKEY','JJ']
school_data = reduce(lambda left,right: pd.merge(left,right,on=merge_cols,how='left'), crdc_dfs)

In [84]:
# keep only schools that have Grade 12
high_schools = school_data[school_data['SCH_GRADE_G12']]

In [85]:
# remove juvenile justice facilities
high_schools = high_schools[high_schools.JJ.eq(False)]
high_schools = high_schools.drop(columns='JJ')

In [86]:
# get rid of non-high-school data and calculated tot columns
import re
high_schools = high_schools.drop(columns=[col for col in high_schools.columns if bool(re.match(r'.*_(G0[1-8]|KG|PS|UG).*', col))])
high_schools = high_schools.drop(columns=[col for col in high_schools.columns if bool(re.match(r'^TOT.*', col))])

In [87]:
# sort columns by percentage NA
(high_schools.isna().sum()/high_schools.shape[0]).sort_values(ascending=False)

SCH_HBREPORTED_DIS_BL_F    1.0
SCH_RS_IDEA_PHYS_AS_F      1.0
SCH_RS_IDEA_MECH_TR_F      1.0
SCH_RS_IDEA_MECH_TR_M      1.0
SCH_RS_IDEA_MECH_WH_F      1.0
                          ... 
SCH_GRADE_G12              0.0
SCH_GRADE_G11              0.0
SCH_GRADE_G09              0.0
LEA_STATE_NAME             0.0
LEA_STATE                  0.0
Length: 1264, dtype: float64

In [88]:
# drop columns with no data
high_schools = high_schools.dropna(axis=1, how='all')
high_schools.shape

(24473, 835)

In [89]:
# drop columns with more than 40% na vals
high_schools = high_schools.dropna(axis=1, thresh=int(high_schools.shape[0]*(1-.4)))
high_schools.shape

(24473, 529)

In [90]:
# identify remaining indicator fields
high_schools.filter(regex='_IND')

,SCH_APENR_IND,SCH_CORPINSTANCES_IND,SCH_CREDITRECOVERY_IND,SCH_DUAL_IND,SCH_GT_IND,SCH_IBENR_IND,SCH_FIREARM_IND,SCH_HOMICIDE_IND,SCH_RET_G09_IND,SCH_RET_G10_IND,SCH_RET_G11_IND,SCH_RET_G12_IND,SCH_SSATHLETICS_IND,SCH_SSCLASSES_IND
2,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
5,True,False,True,True,False,False,False,False,True,True,True,True,True,False
11,True,True,True,True,False,False,False,False,True,True,True,True,True,False
14,True,True,True,True,False,False,False,False,True,True,True,True,True,False
16,True,True,True,True,False,False,False,False,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97617,True,False,False,False,False,False,False,False,False,False,False,False,False,False
97620,True,False,False,False,False,False,False,False,True,True,True,False,False,False
97621,False,False,False,False,False,False,False,False,True,True,True,False,False,False
97623,True,False,False,False,False,False,False,False,True,True,False,False,False,False


In [91]:
# get rid of duplicated data from indicators
high_schools.drop(columns=['SCH_APENR_IND','SCH_CORPINSTANCES_IND','SCH_CREDITRECOVERY_IND','SCH_DUAL_IND','SCH_GT_IND','SCH_IBENR_IND','SCH_RET_G09_IND','SCH_RET_G10_IND','SCH_RET_G11_IND','SCH_RET_G12_IND','SCH_SSCLASSES_IND','SCH_SSCLASSES_IND'], inplace=True)

In [92]:
# treat bools as floats
high_schools = high_schools.replace(True, 1)
high_schools = high_schools.replace(False, 0)

In [93]:
high_schools.describe()

,SCH_MATHCLASSES_ADVM,SCH_MATHCERT_ADVM,SCH_MATHENR_ADVM_HI_M,SCH_MATHENR_ADVM_HI_F,SCH_MATHENR_ADVM_AM_M,SCH_MATHENR_ADVM_AM_F,SCH_MATHENR_ADVM_AS_M,SCH_MATHENR_ADVM_AS_F,SCH_MATHENR_ADVM_HP_M,SCH_MATHENR_ADVM_HP_F,...,SCH_DISCWDIS_TFRALT_IDEA_BL_M,SCH_DISCWDIS_TFRALT_IDEA_BL_F,SCH_DISCWDIS_TFRALT_IDEA_WH_M,SCH_DISCWDIS_TFRALT_IDEA_WH_F,SCH_DISCWDIS_TFRALT_IDEA_TR_M,SCH_DISCWDIS_TFRALT_IDEA_TR_F,SCH_DISCWDIS_TFRALT_LEP_M,SCH_DISCWDIS_TFRALT_LEP_F,SCH_DISCWDIS_TFRALT_504_M,SCH_DISCWDIS_TFRALT_504_F
count,24469.000000,17534.000000,17534.000000,17534.000000,17534.000000,17534.000000,17534.000000,17534.000000,17534.000000,17534.000000,...,24467.000000,24467.000000,24467.000000,24467.000000,24467.000000,24467.000000,24471.000000,24471.000000,24468.000000,24467.000000
mean,5.655932,6.675488,13.579389,14.926429,0.346641,0.418729,5.473252,5.656154,0.186096,0.220657,...,0.101892,0.027425,0.075816,0.017411,0.006458,0.002166,0.013853,0.002207,0.035475,0.010954
std,9.564453,9.356777,32.233838,34.572369,1.680068,2.013496,20.738184,20.642909,1.506309,1.976380,...,0.889207,0.350905,0.519872,0.186456,0.093301,0.049886,0.199745,0.056415,0.391557,0.174976
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,4.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,9.000000,11.000000,13.000000,0.000000,0.000000,3.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,608.000000,608.000000,571.000000,552.000000,70.000000,69.000000,587.000000,564.000000,103.000000,138.000000,...,64.000000,30.000000,15.000000,6.000000,5.000000,2.000000,12.000000,4.000000,17.000000,9.000000


In [94]:
# add NC ACT data
nc_act = pd.read_excel('../data/act-results/NCactresults1718.xlsx',header=2,usecols=[0,1,2,3,4],na_values=[' ---- ','*'])
nc_act.dropna(inplace=True)
nc_act = nc_act[['System or School Name','Mean']]
nc_act.columns = ['SCH_NAME','ACT']
nc_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('NC')], nc_act, on='SCH_NAME')

In [95]:
# add SC ACT data
sc_act = pd.read_excel('../data/act-results/SCACT-Schools2017final2.xlsx',header=2,usecols=[0,1,2,10])
sc_act.dropna(inplace=True)
sc_act = sc_act[['Unnamed: 2',18.4]]
sc_act.columns = ['SCH_NAME','ACT']
high_schools.loc[high_schools['LEA_STATE'].eq('SC'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('SC')]['SCH_NAME'].str.upper()
sc_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('SC')], sc_act, on='SCH_NAME')

In [96]:
# add GA ACT data
ga_act = pd.read_csv('../data/act-results/GA_ACT_HIGHEST_2018_FEB_24_2020.csv',usecols=[1,4,6,14])
ga_act.dropna(inplace=True)
ga_act = ga_act[ga_act['TEST_CMPNT_TYP_CD'].eq('Composite')]
ga_act = ga_act[['INSTN_NAME','INSTN_AVG_SCORE_VAL']]
ga_act.columns = ['SCH_NAME','ACT']
ga_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('GA')], ga_act, on='SCH_NAME')

In [97]:
# add CO
co_act = pd.read_excel('../data/act-results/CO_ACT District and School Summary 2015_2016.xlsx',header=4,usecols=[3,11])
co_act.dropna(inplace=True)
co_act.columns = ['SCH_NAME','ACT']
high_schools.loc[high_schools['LEA_STATE'].eq('CO'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('CO')]['SCH_NAME'].str.upper()
co_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('CO')], co_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('CO')].shape, co_high_schools.shape

((494, 518), (431, 519))

In [98]:
# add FL
fl_act = pd.read_excel('../data/act-results/FL_2017ACTSchool.xlsx',header=3,usecols=[3,9],na_values=['*'])
fl_act.dropna(inplace=True)
fl_act['Unnamed: 3'] = fl_act['Unnamed: 3'].str.replace(' HS',' HIGH SCHOOL')
fl_act.columns = ['SCH_NAME','ACT']
high_schools.loc[high_schools['LEA_STATE'].eq('FL'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('FL')]['SCH_NAME'].str.upper()
fl_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('FL')], fl_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('FL')].shape, fl_high_schools.shape

((951, 518), (286, 519))

In [99]:
# add LA
la_act = pd.read_excel('../data/act-results/LA_act-class-of-2018.xlsx',sheet_name='Site',header=5,usecols=[3,5])
la_act.dropna(inplace=True)
la_act.columns = ['SCH_NAME','ACT']
la_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('LA')], la_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('LA')].shape, la_high_schools.shape

((353, 518), (341, 519))

In [100]:
mo_act = pd.read_excel('../data/act-results/MO_Building ACT Results.xlsx',header=0,usecols=[0,4,9])
mo_act.dropna(inplace=True)
mo_act = mo_act[mo_act['YEAR'].eq(2018)].iloc[:,1:]
mo_act.columns = ['SCH_NAME','ACT']
high_schools.loc[high_schools['LEA_STATE'].eq('MO'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('MO')]['SCH_NAME'].str.upper()
mo_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('MO')], mo_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('MO')].shape, mo_high_schools.shape

((639, 518), (576, 519))

In [114]:
mn_act = pd.read_excel('../data/act-results/Minnesota 2018 Public Schools Graduating Class 5 Year Trends.xlsx',header=0,usecols=[0,3,5,11])
mn_act.dropna(inplace=True)
mn_act = mn_act[mn_act['Analysis Level'].eq('School') & mn_act['Grad Year'].eq(2018)]
mn_act = mn_act[['HS Name','Avg Comp']]
mn_act.columns = ['SCH_NAME','ACT']
high_schools.loc[high_schools['LEA_STATE'].eq('MN'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('MN')]['SCH_NAME'].str.upper()
high_schools.loc[high_schools['LEA_STATE'].eq('MN'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('MN')]['SCH_NAME'].str.replace('SECONDARY','HIGH SCHOOL')
high_schools.loc[high_schools['LEA_STATE'].eq('MN'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('MN')]['SCH_NAME'].str.replace('SENIOR HIGH','HIGH SCHOOL')
mn_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('MN')], mn_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('MN')].shape, mn_high_schools.shape

((816, 518), (215, 519))

In [116]:
ne_act = pd.read_csv('../data/act-results/NE_ACT_Composite_20202021.csv')
ne_act.dropna(inplace=True)
ne_act = ne_act[ne_act['LEVEL'].eq('SC')]
ne_act = ne_act[['NAME','AVERAGE_COMPOSITE_SCORE']]
ne_act.columns = ['SCH_NAME','ACT']
high_schools.loc[high_schools['LEA_STATE'].eq('NE'),'SCH_NAME'] = high_schools[high_schools['LEA_STATE'].eq('NE')]['SCH_NAME'].str.upper()
ne_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('NE')], ne_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('NE')].shape, ne_high_schools.shape

((312, 518), (263, 519))

In [122]:
nd_act = pd.read_excel('../data/act-results/North Dakota ACT Scores.xlsx',header=1,usecols=[1,10])
nd_act.dropna(inplace=True)
nd_act['School'] = nd_act['School'].str.upper()
nd_act.columns = ['SCH_NAME','ACT']
nd_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('ND')], nd_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('ND')].shape, nd_high_schools.shape

((167, 518), (88, 519))

In [123]:
ok_act = pd.read_excel('../data/act-results/OK_School Level_ACT_2016_Senior.xlsx',header=0,usecols=[5,7])
ok_act.dropna(inplace=True)
ok_act.columns = ['SCH_NAME',"ACT"]
ok_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('OK')], ok_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('OK')].shape, ok_high_schools.shape

((490, 518), (458, 519))

In [121]:
pa_act = pd.read_excel('../data/act-results/Pennsylvania_2017 ACT Scores Public Schools.xlsx',header=7,usecols=[4,11])
pa_act.dropna(inplace=True)
pa_act.columns = ['SCH_NAME','ACT']
pa_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('PA')], pa_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('PA')].shape, pa_high_schools.shape

((751, 518), (565, 519))

In [119]:
tn_act = pd.read_excel('../data/act-results/TN_2017-18 ACT_school_suppressed.xlsx',header=0,usecols=[3,4,11])
tn_act.dropna(inplace=True)
tn_act = tn_act[tn_act['Subgroup'].eq('All Students')][['School Name','Average Composite Score']]
tn_act.columns = ['SCH_NAME','ACT']
tn_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('TN')], tn_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('TN')].shape, tn_high_schools.shape

((402, 518), (395, 519))

In [120]:
tx_act = pd.read_excel('../data/act-results/TX_ACT_Campus_Data_Class_2018.xlsx',sheet_name='ACT_Campus_Data_Class_2018',header=0,usecols=[0,3,14])
tx_act.dropna(inplace=True)
tx_act = tx_act[tx_act['Group'].eq('All Students')].iloc[:,1:]
tx_act['CampName'] = tx_act['CampName'].str.upper()
tx_act.columns = ['SCH_NAME','ACT']
tx_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('TX')], tx_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('TX')].shape, tx_high_schools.shape

((2033, 518), (1605, 519))

In [124]:
ut_act = pd.read_excel('../data/act-results/Utah_ACTGrade112018.xlsx',sheet_name='SCHOOL',header=0,usecols=[3,5])
ut_act.dropna(inplace=True)
ut_act.columns = ['SCH_NAME','ACT']
ut_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('UT')], ut_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('UT')].shape, ut_high_schools.shape

((215, 518), (186, 519))

In [126]:
wi_act = pd.read_csv('../data/act-results/WI_act_graduates_certified_2017-18.csv')
wi_act = wi_act[wi_act['TEST_SUBJECT'].eq('Composite') & wi_act['GROUP_BY'].eq('All Students')]
wi_act = wi_act[['SCHOOL_NAME','AVERAGE_SCORE']]
wi_act.columns = ['SCH_NAME','ACT']
wi_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('WI')], wi_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('WI')].shape, wi_high_schools.shape

((576, 518), (682, 519))

In [135]:
wy_act = pd.read_excel('../data/act-results/WY_DistrictACTSuiteAverages_20220329_050233.xlsx',header=8,usecols=[4,20])
wy_act.dropna(inplace=True)
wy_act.columns = ['SCH_NAME','ACT']
wy_high_schools = pd.merge(high_schools[high_schools['LEA_STATE'].eq('WY')], wy_act, on='SCH_NAME')
high_schools[high_schools['LEA_STATE'].eq('WY')].shape, wy_high_schools.shape

C:\Users\David\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


((95, 518), (70, 519))

In [136]:
high_schools_act = pd.concat([nc_high_schools,sc_high_schools,ga_high_schools,co_high_schools,fl_high_schools,la_high_schools,mn_high_schools,mo_high_schools,ne_high_schools,nd_high_schools,ok_high_schools,pa_high_schools,tn_high_schools,tx_high_schools,ut_high_schools,wi_high_schools,wy_high_schools])

In [137]:
# replace missing data with either mean or median depending on skewness
from scipy.stats import skew

for i, col in enumerate(high_schools_act.columns):
    if high_schools_act[col].dtype == object:
        continue
    sk = skew(high_schools_act[col], nan_policy='omit')
    if abs(sk) > 1:
        high_schools_act[col].fillna(high_schools_act[col].median(), inplace=True)
    else:
        high_schools_act[col].fillna(high_schools_act[col].mean(), inplace=True)


In [138]:
high_schools_act.isna().sum().sort_values(ascending=False)

LEA_STATE                       0
SCH_DISCWDIS_EXPWE_LEP_M        0
SCH_DISCWDIS_EXPZT_IDEA_BL_M    0
SCH_DISCWDIS_EXPZT_IDEA_HP_F    0
SCH_DISCWDIS_EXPZT_IDEA_HP_M    0
                               ..
SCH_DUALENR_AS_F                0
SCH_DUALENR_AS_M                0
SCH_DUALENR_AM_F                0
SCH_DUALENR_AM_M                0
ACT                             0
Length: 519, dtype: int64

In [141]:
high_schools_act.to_csv('../data/full_highschool_data.csv',index=False)